In [10]:
import KSEM
import importlib
importlib.reload(KSEM)

from KSEM import *

In [2]:
Fgate = np.array([[1,0],[0,1]], dtype=complex)
Fsup = np.kron(Fgate.conjugate(), Fgate)
ys = [np.array([0.95, 0.05],dtype=np.float64),
     np.array([0.9, 0.1],dtype=np.float64),
     np.array([0.8, 0.2],dtype=np.float64)]
x = np.array([0.99,0,0,0.01], dtype=complex).reshape((4,1)) # matrix -> vector, numpy.reshape(, order='F')

M = np.identity(4, dtype=complex)* 0.02 * (1) # a guess for covariance matrix, E[(x0-xhat0^+)(x0-xhat0^+)^T]
Q = np.identity(4, dtype=complex)* 0.2 * (1) # state covariance
R = np.identity(2, dtype=complex)* 0.1 * (1) # meas covariance
P = np.identity(4, dtype=complex)* 0.05 * (1)# 

In [3]:
x.shape, ys[0].shape, Fsup.shape

((4, 1), (2,), (4, 4))

In [4]:
M.shape, Q.shape, R.shape, P.shape

((4, 4), (4, 4), (2, 2), (4, 4))

In [5]:
smthr = KSQS(ys, Fsup, x, M, Q, R, P)
x_seq, M_seq, M_prio_seq = smthr.smooth()

/home/firepanda415/anaconda3/envs/quantum/lib/python3.9/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/home/firepanda415/anaconda3/envs/quantum/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/firepanda415/anaconda3/envs/quantum/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


In [6]:
x_seq_np = []
for i in range(len(x_seq)):
    x_seq_np.append(x_seq[i].toarray().flatten())
    
x_seq_np

[array([0.98664445+0.j, 0.        +0.j, 0.        +0.j, 0.01335555+0.j,
        0.98664445+0.j, 0.        +0.j, 0.        +0.j, 0.01335555+0.j]),
 array([0.94470005+0.j, 0.        +0.j, 0.        +0.j, 0.05529995+0.j,
        0.94470005+0.j, 0.        +0.j, 0.        +0.j, 0.05529995+0.j]),
 array([0.89613071+0.j, 0.        +0.j, 0.        +0.j, 0.10386929+0.j,
        0.89613071+0.j, 0.        +0.j, 0.        +0.j, 0.10386929+0.j]),
 array([0.84272476+0.j, 0.        +0.j, 0.        +0.j, 0.15727524+0.j,
        0.84272476+0.j, 0.        +0.j, 0.        +0.j, 0.15727524+0.j])]

In [7]:
M_seq_np = []
for j in range(len(M_seq)):
    print(np.linalg.norm(M_seq[j].toarray()[range(4),:][:,range(4)].imag))
    M_seq_np.append( M_seq[j].toarray()[range(4),:][:,range(4)].real)
    
M_seq_np

0.0
0.0
0.0
0.0


[array([[0.01950252, 0.        , 0.        , 0.        ],
        [0.        , 0.02      , 0.        , 0.        ],
        [0.        , 0.        , 0.02      , 0.        ],
        [0.        , 0.        , 0.        , 0.01950252]]),
 array([[0.12933425, 0.        , 0.        , 0.        ],
        [0.        , 0.22      , 0.        , 0.        ],
        [0.        , 0.        , 0.22      , 0.        ],
        [0.        , 0.        , 0.        , 0.12933425]]),
 array([[0.21018696, 0.        , 0.        , 0.        ],
        [0.        , 0.42      , 0.        , 0.        ],
        [0.        , 0.        , 0.42      , 0.        ],
        [0.        , 0.        , 0.        , 0.21018696]]),
 array([[0.30046903, 0.        , 0.        , 0.        ],
        [0.        , 0.62      , 0.        , 0.        ],
        [0.        , 0.        , 0.62      , 0.        ],
        [0.        , 0.        , 0.        , 0.30046903]])]

In [11]:
num_dim_state = 4
num_dim_obs = 2
learn_obj = EMLearn(ys, Fsup, x, M, Q, R, P)
estX0, estM0, estQ, estR, estF = learn_obj.learn() # they are all arguemented
realM0 = estM0.toarray()[range(num_dim_state),:][:,range(num_dim_state)]
realF = estF.toarray()[range(num_dim_state),:][:,range(num_dim_state)]
realQ = estQ.toarray()[range(num_dim_state),:][:,range(num_dim_state)]
realR = estR.toarray()[range(num_dim_obs),:][:,range(num_dim_obs)]
realP = estQ.toarray()[range(num_dim_state),:][:,range(num_dim_state, 2*num_dim_state)]

Iteration statrts New ll 26.18, Last ll 16.18
Iteration     1, New log-likelihood 3.00913e+01, Last log-likelihood 2.61808e+01, Change 3.91056e+00
Iteration     2, New log-likelihood 3.38249e+01, Last log-likelihood 3.00913e+01, Change 3.73359e+00
Iteration     3, New log-likelihood 3.73339e+01, Last log-likelihood 3.38249e+01, Change 3.50896e+00
Iteration     4, New log-likelihood 4.05819e+01, Last log-likelihood 3.73339e+01, Change 3.24806e+00
Iteration     5, New log-likelihood 4.35568e+01, Last log-likelihood 4.05819e+01, Change 2.97487e+00
Iteration     6, New log-likelihood 4.62756e+01, Last log-likelihood 4.35568e+01, Change 2.71880e+00
Iteration     7, New log-likelihood 4.87825e+01, Last log-likelihood 4.62756e+01, Change 2.50693e+00
Iteration     8, New log-likelihood 5.11335e+01, Last log-likelihood 4.87825e+01, Change 2.35096e+00
Iteration     9, New log-likelihood 5.33790e+01, Last log-likelihood 5.11335e+01, Change 2.24554e+00
Iteration    10, New log-likelihood 5.55566e+

In [12]:
realF

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [13]:
realQ.real, np.linalg.norm(realQ.imag)

(array([[ 0.05808681,  0.        ,  0.        , -0.01342807],
        [ 0.        ,  0.2       ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.2       ,  0.        ],
        [-0.01342807,  0.        ,  0.        ,  0.07806119]]),
 0.0)

In [14]:
realR.real, np.linalg.norm(realR.imag)

(array([[ 0.00110675, -0.00107908],
        [-0.00107908,  0.00110675]]),
 0.0)

In [15]:
realP.real, np.linalg.norm(realP.imag)

(array([[-0.07264109,  0.        ,  0.        , -0.01415848],
        [ 0.        ,  0.05      ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.05      ,  0.        ],
        [ 0.00578982,  0.        ,  0.        , -0.07266716]]),
 0.0)

In [16]:
smthr2 = KSQS(ys, realF, x, realM0, realQ, realR, realP)
x_seq2, M_seq2, M_prio_seq2 = smthr2.smooth() 
x_seq_np2 = []
for i in range(len(x_seq2)):
    x_seq_np2.append(x_seq2[i].toarray().flatten())
    
x_seq_np2

[array([0.96826674+0.j, 0.        +0.j, 0.        +0.j, 0.03172358+0.j,
        0.96826674+0.j, 0.        +0.j, 0.        +0.j, 0.03172358+0.j]),
 array([0.94224591+0.j, 0.        +0.j, 0.        +0.j, 0.05775412+0.j,
        0.94224591+0.j, 0.        +0.j, 0.        +0.j, 0.05775412+0.j]),
 array([0.8931002+0.j, 0.       +0.j, 0.       +0.j, 0.1068998+0.j,
        0.8931002+0.j, 0.       +0.j, 0.       +0.j, 0.1068998+0.j]),
 array([0.82338564+0.j, 0.        +0.j, 0.        +0.j, 0.17661436+0.j,
        0.82338564+0.j, 0.        +0.j, 0.        +0.j, 0.17661436+0.j])]